In [5]:
#import stuff
import torch
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

# import packages
from typing import Dict, List
from detectron2.data.datasets import register_coco_instances
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import sys


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode

In [6]:
#easy registering of dataset since it is in COCO format
register_coco_instances("tdt4265_dataset_train", {}, "./tdt4265_2022/train_annotations.json", "./tdt4265_2022") #get the datasets, easy since thet are in coco format
register_coco_instances("tdt4265_dataset_val", {}, "./tdt4265_2022/val_annotations.json", "./tdt4265_2022")



In [26]:

#access the data and metadata
from detectron2.data import DatasetCatalog

#to access the data:
trainDataDict: List[Dict] = DatasetCatalog.get("tdt4265_dataset_train")
valDataDict: List[Dict] = DatasetCatalog.get("tdt4265_dataset_val")
trainMetaData = MetadataCatalog.get("tdt4265_dataset_train")
valMetaData = MetadataCatalog.get("tdt4265_dataset_val")



[04/30 19:11:07 d2.data.datasets.coco]: Loaded 1604 images in COCO format from ./tdt4265_2022/train_annotations.json
[04/30 19:11:07 d2.data.datasets.coco]: Loaded 301 images in COCO format from ./tdt4265_2022/val_annotations.json


In [27]:
#to view images from the tdt4265 dataset 
def showImagesWithAnnotations(numberOfImages : int, dataset : str):
    if dataset.lower() == "train":
        dataDict: List[Dict] = DatasetCatalog.get("tdt4265_dataset_train")
    elif dataset.lower() == "val":
        dataDict: List[Dict] = DatasetCatalog.get("tdt4265_dataset_val")
    else:
        raise Exception("Have to choose either 'train' or 'val'")

    dataset_dicts = dataDict
    for d in random.sample(dataset_dicts, numberOfImages):
        #print("D IS:", d)
        #print("VALUE IS:", d["file_name"])
    
        img = cv2.imread(d["file_name"])

        visualizer = Visualizer(img[:, :, ::-1], metadata=trainMetaData, scale=0.5)
        out = visualizer.draw_dataset_dict(d)
        cv2.imshow("image", out.get_image()[:, :, ::-1])
        cv2.waitKey(0) 
        #closing all open windows 
        cv2.destroyAllWindows()  #must have these to view locally in cv2 (in google colab i wouldnt need them)
showImagesWithAnnotations(1, "train")

[04/30 19:11:10 d2.data.datasets.coco]: Loaded 1604 images in COCO format from ./tdt4265_2022/train_annotations.json
D IS: {'file_name': './tdt4265_2022\\images/train/trip007_glos_Video00012_58.png', 'height': 128, 'width': 1024, 'image_id': 1048, 'annotations': [{'iscrowd': 0, 'bbox': [53.1, 80.42, 68.6, 47.58], 'category_id': 0, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [475.65, 70.86, 19.75, 25.75], 'category_id': 0, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [493.19, 68.21, 9.829999999999984, 12.460000000000008], 'category_id': 0, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [507.46, 73.72, 23.010000000000048, 24.960000000000008], 'category_id': 0, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [496.94, 87.38, 43.660000000000025, 39.84], 'category_id': 0, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [400.01, 71.31, 16.980000000000018, 12.950000000000003], 'category_id': 0, 'bbox_mode': <BoxMode.XYWH_ABS: 

In [7]:
#Train the model by running this cell
from detectron2.engine import DefaultTrainer


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("tdt4265_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 5e-3 #same lr as in other models  
cfg.SOLVER.MAX_ITER = 10000    # same number of steps as original model
cfg.SOLVER.STEPS = []        # could decay lr here
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9  #9 classes which which the tdt4265 dataset has

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train() #if cfg not found, comment out this, and run the cell (so it finds the cfg, but doesnt train)

[05/02 19:06:09 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (10, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (10,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (36, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (36,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

In [8]:
# Running inference 
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a iou threshhold to what I used in other models
predictor = DefaultPredictor(cfg)

In [9]:
#register custom dataset:

register_coco_instances("myCustomValDataset", {}, "./customDataset/val_annotations.json", "./customDataset") #if you register several times, it will complain, just register once, and then comment it out

customValDataDict: List[Dict] = DatasetCatalog.get("myCustomValDataset")
customValMetaData = MetadataCatalog.get("myCustomValDataset")

[05/02 19:06:15 d2.data.datasets.coco]: Loaded 4 images in COCO format from ./customDataset/val_annotations.json


In [10]:
#view custom pictures inferenced by the model

for d in random.sample(customValDataDict, 4): 
    #print("D IS:", d)   
    im = cv2.imread(d["file_name"])
    if (im is None):
        print("COULD NOT READ", d["file_name"])
        continue
    outputs = predictor(im)

    v = Visualizer(im[:, :, ::-1],
                    metadata=customValMetaData, 
                    scale=1.2, 
                    instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    cv2.imshow("image", out.get_image()[:, :, ::-1])
    cv2.waitKey(0) 
    #closing all open windows 
    cv2.destroyAllWindows() 

NameError: name 'ColorMode' is not defined

In [34]:
#view inferenced images from tdt4265 val dataset:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = valDataDict
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   #metadata=valMetaData, 
                   scale=1.7, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("image", out.get_image()[:, :, ::-1])
    cv2.waitKey(0) 
    #closing all open windows 
    cv2.destroyAllWindows() 

In [ ]:
#running mAP tests
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader


evaluator = COCOEvaluator("tdt4265_dataset_val", output_dir="./output", tasks=("bbox",), distributed=True)
val_loader = build_detection_test_loader(cfg, "tdt4265_dataset_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))


In [12]:
#make a inferenced video!
import cv2
cap = cv2.VideoCapture('./testVideo4.mp4')

if (cap.isOpened()== False): 
  raise Exception("Error opening video  file")

# Read until video is completed
frameArray = []
while(cap.isOpened()):
      
  # Capture frame-by-frame
  ret, frame = cap.read()
  counter = 0

  if ret == True:
       
        outputs = predictor(frame)  
        v = Visualizer(frame[:, :, ::-1],
                      metadata=customValMetaData,
                      scale=1.7, 
                      instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        frame = out.get_image()[:, :, ::-1]
        #cv2.imshow("Frame:", frame)
        #if cv2.waitKey(25) & 0xFF == ord('q'): #hold q to exit so you dont have to go through every frame
        #  break
        #cv2.waitKey(0) 
        #cv2.destroyAllWindows() 

        frameArray.append(frame)
  else: 
    print("BREAKA")
    break
   

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
height,width,layers=frameArray[0].shape
video = cv2.VideoWriter('inferencedVideo4.mp4', fourcc, 1, (width, height))


for i in range(len(frameArray)):
      video.write(frameArray[i])

cv2.destroyAllWindows()
video.release()



BREAKA


fatal: destination path 'detectron2' already exists and is not an empty directory.


In [56]:
#inference on custom video:

#make a inferenced video!
import cv2
cap = cv2.VideoCapture('./testVideo3.mp4')

if (cap.isOpened()== False): 
  raise Exception("Error opening video  file")

# Read until video is completed
frameArray = []
while(cap.isOpened()):
      
  # Capture frame-by-frame
  ret, frame = cap.read()
  counter = 0

  if ret == True:
       
        outputs = predictor(frame)  
        v = Visualizer(frame[:, :, ::-1],
                      metadata=customValMetaData,
                      scale=1.7, 
                      instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        frame = out.get_image()[:, :, ::-1]
        #cv2.imshow("Frame:", frame)
        #if cv2.waitKey(25) & 0xFF == ord('q'): #hold q to exit so you dont have to go through every frame
        #  break
        #cv2.waitKey(0) 
        #cv2.destroyAllWindows() 

        frameArray.append(frame)
  else: 
    print("BREAKA")
    break
   

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
height,width,layers=frameArray[0].shape
video = cv2.VideoWriter('inferencedVideo2.mp4', fourcc, 1, (width, height))


for i in range(len(frameArray)):
      video.write(frameArray[i])

cv2.destroyAllWindows()
video.release()



BREAKA


In [ ]:
#extra stuff
"""
#Running inference on custom images. Works pretty well, even works for all image shapes (not just 1024 width, 128 height), but does not show labels
from detectron2.utils.visualizer import ColorMode


#im = cv2.imread("./try1img.jpg")
im = cv2.imread("./try2img.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                #metadata=valMetaData, 
                scale=1.2, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2.imshow("image", out.get_image()[:, :, ::-1])
cv2.waitKey(0) 
#closing all open windows 
cv2.destroyAllWindows() 
"""